**Operations Research in Action &#x25aa; Fall 2024**

# Project 2 &ndash; Model and Results &ndash; Part 3

Let's start with some data wrangling to prepare the data for use in an optimization model.

__Read in the player ratings data.__

In [1]:
# Solution
import pandas as pd

In [2]:
# Solution
ratings_df = pd.read_csv('data/player_data.csv')
ratings_df.head()

,player,rating
0,USA_p1,3.0
1,USA_p2,3.0
2,USA_p3,3.5
3,USA_p4,0.0
4,USA_p5,2.0


__Read in the estimated coefficients that we wrote to a CSV file at the end of Part 2. When reading in the data, set the column name for the coefficients to `apm`.__

In [3]:
# Solution
coefs_df = pd.read_csv(
    'coefs.csv',
    skiprows=1,
    header=None,
    names=['player', 'apm']
)
coefs_df.head()

,player,apm
0,(Intercept),0.093834
1,USA_p1,0.359096
2,USA_p2,0.035917
3,USA_p3,0.446143
4,USA_p4,-0.699490


__Merge these two DataFrames by player.__

Hint: the APM for Chile_p12 will be NA, so you'll have to fill in this one manually. To replace all values of NA in a DataFrame with 0, you can apply `.fillna(0)` to the DataFrame.

In [4]:
# Solution
players_df = ratings_df.merge(coefs_df, on=['player'], how='left').fillna(0)
players_df.head()

,player,rating,apm
0,USA_p1,3.0,0.359096
1,USA_p2,3.0,0.035917
2,USA_p3,3.5,0.446143
3,USA_p4,0.0,-0.699490
4,USA_p5,2.0,-0.284978


Now, let's find the lineup for Canada with the maximum total APM.

__On a sheet of paper, write down your optimization model.__

Don't forget to define all the components:

- Sets
- Parameters
- Decision variables
- Objective function
- Constraints

_Solution._

_Sets._

- $P$ = set of players 
    - Let's use Canada as an example

_Parameters._

- $r_i$ = rating for player $i$, for $i \in P$
- $a_i$ = adjusted plus-minus for player $i$, for $i \in P$

_Decision variables._

$$x_i = \begin{cases} 1 & \text{if player } i \text{ is chosen} \\ 0 & \text{otherwise} \end{cases} \quad \text{for } i \in P$$

_Objective function._

Maximize total APM of players selected
$$
\begin{equation*} 
    \text{maximize} \quad \sum_{i \in P} a_{i} x_{i}
\end{equation*}
$$

_Constraints._

Exactly 4 players must be chosen
$$
\begin{equation*}
\text{subject to} \quad \sum_{i \in P} x_{i} = 4
\end{equation*}
$$

Total rating of players selected cannot exceed 8
$$
\begin{equation*}
\sum_{i \in P} r_i x_i \le 8
\end{equation*}
$$

__Now that you've written down your model, implement it using Pyomo.__

In [5]:
# Solution
import pyomo.environ as pyo

In [6]:
# Solution
P = [i for i in players_df['player'] if i.startswith('Canada')]

r = dict(zip(players_df['player'], players_df['rating']))
a = dict(zip(players_df['player'], players_df['apm']))

model = pyo.ConcreteModel()

model.x = pyo.Var(P, domain=pyo.Binary)

def obj_rule(model):
    return sum(a[i] * model.x[i] for i in P)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

def team_size_rule(model):
    return sum(model.x[i] for i in P) == 4
model.team_size = pyo.Constraint(rule=team_size_rule)

def total_rating_rule(model):
    return sum(r[i] * model.x[i] for i in P) <= 8
model.total_rating = pyo.Constraint(rule=total_rating_rule)

solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpkjfdohde.glpk.raw
 --wglp /var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpgy9qdjrn.glpk.glp
 --cpxlp /var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpzotupute.pyomo.lp
Reading problem data from '/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpzotupute.pyomo.lp'...
/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpzotupute.pyomo.lp:63: warning: lower bound of variable 'x2' redefined
/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpzotupute.pyomo.lp:63: warning: upper bound of variable 'x2' redefined
2 rows, 12 columns, 23 non-zeros
12 integer variables, all of which are binary
75 lines were read
Writing problem data to '/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpgy9qdjrn.glpk.glp'...
54 lines were written
GLPK Integer Optimizer 5.0
2 rows, 12 columns, 23 non-zeros
12 integer variables, all of which are binary
Preprocessing...
2 

__Interpret the optimal solution: what is the optimal lineup?__

In [7]:
# Solution
for i in P:
    if model.x[i].value > 0.5:
        print(i)

Canada_p2
Canada_p8
Canada_p9
Canada_p11


__Find the All-Star Team: find the lineup with the maximum total APM across *all* teams.__

In [8]:
# Solution
P = list(players_df['player'])
r = dict(zip(players_df['player'], players_df['rating']))
a = dict(zip(players_df['player'], players_df['apm']))

model = pyo.ConcreteModel()
model.x = pyo.Var(P, domain=pyo.Binary)

def obj_rule(model):
    return sum(a[i] * model.x[i] for i in P)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

def team_size_rule(model):
    return sum(model.x[i] for i in P) == 4
model.team_size = pyo.Constraint(rule=team_size_rule)

def total_rating_rule(model):
    return sum(r[i] * model.x[i] for i in P) <= 8
model.total_rating = pyo.Constraint(rule=total_rating_rule)

solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

for i in P:
    if model.x[i].value > 0.5:
        print(i)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpcvofia2v.glpk.raw
 --wglp /var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmp8ex1fvpq.glpk.glp
 --cpxlp /var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpigdu04fu.pyomo.lp
Reading problem data from '/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpigdu04fu.pyomo.lp'...
/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpigdu04fu.pyomo.lp:585: warning: lower bound of variable 'x2' redefined
/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmpigdu04fu.pyomo.lp:585: warning: upper bound of variable 'x2' redefined
2 rows, 144 columns, 282 non-zeros
144 integer variables, all of which are binary
729 lines were read
Writing problem data to '/var/folders/5f/9j0fjx714bj54dg_j7b_5r_m0000gq/T/tmp8ex1fvpq.glpk.glp'...
576 lines were written
GLPK Integer Optimizer 5.0
2 rows, 144 columns, 282 non-zeros
144 integer variables, all of which are binary
Preproces